In [ ]:
# default_exp multiplayer

# Multiplayer Git 
> Summary

Initially I used Git on a private project where I was the sole developer.
Amongst the commands related to Git's distributed nature, I needed only *pull*
and *clone* so could I keep the same project in different places.

Later I wanted to publish my code with Git, and include changes from
contributors. I had to learn how to manage projects with multiple developers
from all over the world. Fortunately, this is Git's forte, and arguably its
raison d'être.

### Who Am I? 

Every commit has an author name and email, which is shown by *git log*.
By default, Git uses system settings to populate these fields.
To set them explicitly, type:

In [ ]:
  ! git config --global user.name "John Doe"
  ! git config --global user.email johndoe@example.com



Omit the global flag to set these options only for the current repository.

### Git Over SSH, HTTP 

Suppose you have SSH access to a web server, but Git is not installed. Though
less efficient than its native protocol, Git can communicate over HTTP.

Download, compile and install Git in your account, and create a repository in
your web directory:



In [ ]:
 ! GIT_DIR=proj.git git init
 ! cd proj.git
 ! git --bare update-server-info
 ! cp hooks/post-update.sample hooks/post-update



For older versions of Git, the copy command fails and you should run:



In [ ]:
 ! chmod a+x hooks/post-update



Now you can publish your latest edits via SSH from any clone:



In [ ]:
 ! git push web.server:/path/to/proj.git master



and anybody can get your project with:



In [ ]:
 ! git clone http://web.server/proj.git



### Git Over Anything 

Want to synchronize repositories without servers, or even a network connection?
Need to improvise during an emergency? We've seen <<makinghistory, *git
fast-export* and *git fast-import* can convert repositories to a single file
and back>>. We could shuttle such files back and forth to transport git
repositories over any medium, but a more efficient tool is *git bundle*.

The sender creates a 'bundle':



In [ ]:
 ! git bundle create somefile HEAD



then transports the bundle, +somefile+, to the other party somehow: email,
thumb drive, an *xxd* printout and an OCR scanner, reading bits over the phone,
smoke signals, etc. The receiver retrieves commits from the bundle by typing:



In [ ]:
 ! git pull somefile



The receiver can even do this from an empty repository. Despite its
size, +somefile+ contains the entire original git repository.

In larger projects, eliminate waste by bundling only changes the other
repository lacks. For example, suppose the commit ``1b6d...'' is the most
recent commit shared by both parties:



In [ ]:
 ! git bundle create somefile HEAD ^1b6d



If done frequently, one could easily forget which commit was last sent. The
help page suggests using tags to solve this. Namely, after you send a bundle,
type:



In [ ]:
 ! git tag -f lastbundle HEAD



and create new refresher bundles with:



In [ ]:
 ! git bundle create newbundle HEAD ^lastbundle



### Patches: The Global Currency 

Patches are text representations of your changes that can be easily understood
by computers and humans alike. This gives them universal appeal. You can email a
patch to developers no matter what version control system they're using. As long
as your audience can read their email, they can see your edits. Similarly, on
your side, all you require is an email account: there's no need to setup an online Git repository.

Recall from the first chapter:



In [ ]:
 ! git diff 1b6d > my.patch



outputs a patch which can be pasted into an email for discussion. In a Git
repository, type:



In [ ]:
 ! git apply < my.patch



to apply the patch.

In more formal settings, when author names and perhaps signatures should be
recorded, generate the corresponding patches past a certain point by typing:



In [ ]:
 ! git format-patch 1b6d



The resulting files can be given to *git-send-email*, or sent by hand. You can also specify a range of commits:



In [ ]:
 ! git format-patch 1b6d..HEAD^^



On the receiving end, save an email to a file, then type:



In [ ]:
 ! git am < email.txt



This applies the incoming patch and also creates a commit, including information such as the author.

With a browser email client, you may need to click a button to see the email in its raw original form before saving the patch to a file.

There are slight differences for mbox-based email clients, but if you use one
of these, you're probably the sort of person who can figure them out easily
without reading tutorials!

### Sorry, We've Moved 

After cloning a repository, running *git push* or *git pull* will automatically
push to or pull from the original URL. How does Git do this? The secret lies in
config options created with the clone. Let's take a peek:



In [ ]:
 ! git config --list



The +remote.origin.url+ option controls the source URL; ``origin'' is a nickname
given to the source repository. As with the ``master'' branch convention, we may
change or delete this nickname but there is usually no reason for doing so.

If the original repository moves, we can update the URL via:



In [ ]:
 ! git config remote.origin.url git://new.url/proj.git



The +branch.master.merge+ option specifies the default remote branch in
a *git pull*. During the initial clone, it is set to the current branch of the
source repository, so even if the HEAD of the source repository subsequently
moves to a different branch, a later pull will faithfully follow the
original branch.

This option only applies to the repository we first cloned from, which is
recorded in the option +branch.master.remote+. If we pull in from other
repositories we must explicitly state which branch we want:



In [ ]:
 ! git pull git://example.com/other.git master



The above explains why some of our earlier push and pull examples had no
arguments.

### Remote Branches 

When you clone a repository, you also clone all its branches. You may not have
noticed this because Git hides them away: you must ask for them specifically.
This prevents branches in the remote repository from interfering with
your branches, and also makes Git easier for beginners.

List the remote branches with:



In [ ]:
 ! git branch -r



You should see something like:

 origin/HEAD
 origin/master
 origin/experimental

These represent branches and the HEAD of the remote repository, and can be used
in regular Git commands. For example, suppose you have made many commits, and
wish to compare against the last fetched version. You could search through the
logs for the appropriate SHA1 hash, but it's much easier to type:



In [ ]:
 ! git diff origin/HEAD



Or you can see what the ``experimental'' branch has been up to:



In [ ]:
 ! git log origin/experimental



### Multiple Remotes 

Suppose two other developers are working on our project, and we want to
keep tabs on both. We can follow more than one repository at a time with:



In [ ]:
 ! git remote add other git://example.com/some_repo.git
 ! git pull other some_branch



Now we have merged in a branch from the second repository, and we have
easy access to all branches of all repositories:



In [ ]:
 ! git diff origin/experimental^ other/some_branch~5



But what if we just want to compare their changes without affecting our own
work? In other words, we want to examine their branches without having
their changes invade our working directory. Then rather than pull, run:



In [ ]:
 ! git fetch        # Fetch from origin, the default.
 ! git fetch other  # Fetch from the second programmer.



This just fetches histories. Although the working directory remains untouched,
we can refer to any branch of any repository in a Git command because we now
possess a local copy.

Recall that behind the scenes, a pull is simply a *fetch* then *merge*.
Usually we *pull* because we want to merge the latest commit after a fetch;
this situation is a notable exception.

See *git help remote* for how to remove remote repositories, ignore certain
branches, and more.

### My Preferences 

For my projects, I like contributors to prepare repositories from which I can
pull. Some Git hosting services let you host your own fork of a project with
the click of a button.

After I fetch a tree, I run Git commands to navigate and examine the changes,
which ideally are well-organized and well-described. I merge my own changes,
and perhaps make further edits. Once satisfied, I push to the main repository.

Though I infrequently receive contributions, I believe this approach scales
well. See
http://torvalds-family.blogspot.com/2009/06/happiness-is-warm-scm.html[this
blog post by Linus Torvalds].

Staying in the Git world is slightly more convenient than patch files, as it
saves me from converting them to Git commits. Furthermore, Git handles details
such as recording the author's name and email address, as well as the time and
date, and asks the author to describe their own change.
